## Workshop with SPARK DataFrame

Try by yourself
* Reminder: documentation at https://spark.apache.org/docs/latest/api/python/index.html

## 1. Start session

In [3]:
# Create the SparkSession
# and read the dataset

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("DataFrame Workshop") \
        .config("spark.ui.showConsoleProgress","false") \
        .getOrCreate()

online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"

df = spark.read \
        .option("header", "true") \
        .option("timestampFormat", "M/d/yyyy H:m")\
        .csv("./data/online-retail-dataset.csv.gz",
             schema=online_retail_schema)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/17 15:50:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/17 15:51:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/17 15:51:01 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [4]:
spark

## 2. Show 5 lines of the "description" column

In [5]:
df.select("description").show(5,truncate=False)

+-----------------------------------+
|description                        |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows


## 3. Find out in which month most invoices have been processed

In [6]:
# This shows how many line items have been processed per month

from pyspark.sql.functions import month
df.groupby(month("InvoiceDate")).count().sort("count").show()

+------------------+-----+
|month(InvoiceDate)|count|
+------------------+-----+
|                 2|27707|
|                 4|29916|
|                 1|35147|
|                 8|35284|
|                 3|36748|
|                 6|36874|
|                 5|37030|
|                 7|39518|
|                 9|50226|
|                10|60742|
|                12|68006|
|                11|84711|
+------------------+-----+



In [7]:
# This shows how distinct invoices have been processed per month

from pyspark.sql.functions import col, month, countDistinct

(df
   .groupBy(month('InvoiceDate'))
   .agg(countDistinct('InvoiceNo').alias('DistinctInvoices'))
   .orderBy(col('DistinctInvoices').desc())
   .show()
)

+------------------+----------------+
|month(InvoiceDate)|DistinctInvoices|
+------------------+----------------+
|                11|            3021|
|                12|            2568|
|                10|            2275|
|                 9|            1994|
|                 5|            1848|
|                 6|            1683|
|                 3|            1665|
|                 7|            1657|
|                 4|            1504|
|                 8|            1456|
|                 1|            1216|
|                 2|            1174|
+------------------+----------------+



## 4. Filter the lines where the Quantity is more than 30

In [8]:
df.where("Quantity > 30").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536367|    84879|ASSORTED COLOUR B...|      32|2010-12-01 08:34:00|     1.69|     13047|United Kingdom|
|   536370|    10002|INFLATABLE POLITI...|      48|2010-12-01 08:45:00|     0.85|     12583|        France|
|   536370|    22492|MINI PAINT SET VI...|      36|2010-12-01 08:45:00|     0.65|     12583|        France|
|   536371|    22086|PAPER CHAIN KIT 5...|      80|2010-12-01 09:00:00|     2.55|     13748|United Kingdom|
|   536374|    21258|VICTORIAN SEWING ...|      32|2010-12-01 09:09:00|    10.95|     15100|United Kingdom|
|   536376|    22114|HOT WATER BOTTLE ...|      48|2010-12-01 09:32:00|     3.45|     15291|United Kingdom|
|   536376|    21733|RED HAN

## 5. Show the four most sold items (by quantity)

In [9]:
from pyspark.sql.functions import desc, asc, expr
(df.groupBy("Description")
     .agg(expr("sum(Quantity) as totalQuantity"))
     .sort("totalQuantity", ascending=False)
     .show(4))

+--------------------+-------------+
|         Description|totalQuantity|
+--------------------+-------------+
|WORLD WAR 2 GLIDE...|        53847|
|JUMBO BAG RED RET...|        47363|
|ASSORTED COLOUR B...|        36381|
|      POPCORN HOLDER|        36334|
+--------------------+-------------+
only showing top 4 rows


## 6. Why do these two operations return different results ?

In [10]:
print(df.select("InvoiceNo").distinct().count())

from pyspark.sql.functions import countDistinct
df.select(countDistinct("InvoiceNo")).show()

22062
+-------------------------+
|count(DISTINCT InvoiceNo)|
+-------------------------+
|                    22061|
+-------------------------+



As you can see from the output of countDistinct, internally it runs count(DISTINCT, which excludes nulls.

https://spark.apache.org/docs/latest/api/sql/#count

* count() Returns the total number of retrieved rows, including rows containing null
* count(DISTINCT expr[, expr...]) - Returns the number of rows for which the supplied expression(s) are unique and non-null.